# ERA5 temperature anomaly on pressure levels

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Select time range

In [ ]:
start = "2021-01"
stop = "2022-12"

## Define request

In [ ]:
collection_id = "reanalysis-era5-pressure-levels-monthly-means"
request = {
    "format": "netcdf",
    "product_type": "monthly_averaged_reanalysis",
    "variable": "temperature",
    "pressure_level": [
        "1",
        "2",
        "3",
        "5",
        "7",
        "10",
        "20",
        "30",
        "50",
        "70",
        "100",
        "125",
        "150",
        "175",
        "200",
        "225",
        "250",
        "300",
        "350",
        "400",
        "450",
        "500",
        "550",
        "600",
        "650",
        "700",
        "750",
        "775",
        "800",
        "825",
        "850",
        "875",
        "900",
        "925",
        "950",
        "975",
        "1000",
    ],
    "time": "00:00",
}
requests = download.update_request_date(request, start=start, stop=stop)

## Download data

In [ ]:
ds = download.download_and_transform(
    collection_id,
    requests,
    transform_func=diagnostics.spatial_weighted_mean,
    chunks={"year": 1},
)

## Compute anomalies

In [ ]:
with xr.set_options(keep_attrs=True):
    ds_anoma = ds.groupby("time.month") - ds.groupby("time.month").mean("time")
ds_anoma["t"].attrs["long_name"] = ds["t"].attrs["long_name"] + " Anomaly"

## Hovmöller Diagram

In [ ]:
ds_anoma["level"].attrs.update({"long_name": "Level", "units": "hPa"})
ds_anoma["t"].plot.contourf(
    x="time",
    yscale="log",
    robust=True,
    levels=15,
    yincrease=False,
    figsize=(15, 5),
    cbar_kwargs={"orientation": "horizontal"},
)
yticks = [1000, 500, 100, 10, 1]
plt.yticks(yticks, yticks)
plt.xlabel(None)
plt.title("ERA5 temperature on pressure levels")